In [14]:
# UTILS FUNCTIONS AND GLOBAL VARIABLES

import pandas as pd

STATE_NAMES = ["Alaska", "Alabama", "Arkansas", "Arizona", "California",
"Colorado", "Connecticut", "Delaware", "Florida", "Georgia",
"Hawaii", "Iowa", "Idaho", "Illinois", "Indiana", "Kansas", "Kentucky", "Louisiana",
"Massachusetts", "Maryland", "Maine", "Michigan", "Minnesota", "Missouri", "Mississippi",
"Montana", "North Carolina", "North Dakota", "Nebraska", "New Hampshire", "New Jersey",
"New Mexico", "Nevada", "New York", "Ohio", "Oklahoma", "Oregon", "Pennsylvania",
"Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Virginia",
"Vermont", "Washington", "Wisconsin", "West Virginia", "Wyoming"]

STATE_NAMES_AND_UNITED_STATES = STATE_NAMES[:]
STATE_NAMES_AND_UNITED_STATES.append("United States")

us_state_abbreviations = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
}

US_STATE_CODES = dict(map(reversed, us_state_abbreviations.items()))

NAICS_SECTOR_CODES = {
    "Agriculture, Forestry, Fishing and Hunting" : ("11",),
    "Mining, Quarrying, and Oil and Gas Extraction" : ("21",),
    "Utilities" : ("22",),
    "Construction" : ("23",),
    "Manufacturing" : ("31", "32", "33",),
    "Wholesale Trade" : ("42",),
    "Retail Trade" : ("44", "45",),
    "Transportation and Warehousing" : ("48", "49",),
    "Information" : ("51",),
    "Finance and Insurance" : ("52",),
    "Real Estate and Rental and Leasing" : ("53",),
    "Professional, Scientific, and Technical Services" : ("54",),
    "Management of Companies and Enterprises" : ("55",),
    "Administrative and Support and Waste Management and Remediation Services" : ("56",),
    "Educational Services" : ("61",),
    "Health Care and Social Assistance" : ("62",),
    "Arts, Entertainment, and Recreation" : ("71",),
    "Accommodation and Food Services" : ("72",),
    "Other Services (except Public Administration)" : ("81",),
    "Public Administration (not covered in economic census)" : ("92",)
}

def combine_dataframes_by_state(main_df, df_lst):
    """
    Recursively concatenates multiple panda dataframes (with "State" 
    as the index) with only the required columns

    Inputs:
        df_lst (lst of tuples): (df, [cols to extract])
        ### If extracting all columns, [cols to extract] should be 
        an empty list ###
        
    Returns:
        final_df (pandas series): concatenated pandas dataframes
    """
    if len(df_lst) == 0:
        return main_df

    other_df, col_lst = df_lst.pop()
    if col_lst != []:
        new_df = main_df.merge(other_df[col_lst], on="State")
    else:
        new_df = main_df.merge(other_df, on="State")
    
    return combine_dataframes_by_state(new_df, df_lst)

In [3]:
# CLEANS CENSUS EXPENDITURE

df = pd.read_csv("2020_us_state_finances.csv")
df['Description'] = df['Description'].str.strip()

df.rename(columns = {'United States Total':'United States'}, inplace = True)

# Retains only columns required
df = df[["Description"] + STATE_NAMES_AND_UNITED_STATES]
# df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

# Drops rows relating to revenue sources
df.drop(df.index[0:66], inplace=True)

social = ["Public welfare", "Hospitals", "Health", "Employment security administration", "Veterans' services"]
educ = ["Education", "Libraries"]
govt = ["Financial administration", "Judicial and legal", "General public buildings", "Other governmental administration"]
transport = ["Highways", "Air transportation (airports)", "Parking facilities", "Sea and inland port facilities"]
others = ["Utility expenditure", "Expenditure1"]

# Retains only required rows
df = df[df["Description"].isin(social + educ + govt + transport + others)]

# # Education (Education + Libraries) - 61
# # Health & Social Services (Public welfare + Hospitals + Health + Security + Employment security administration +  Veterans' services) - 62
# # Government Administration (Financial administration + Judicial and legal + General public buildings + Other governmental administration) - 92
# # Utilities (Utility expenditure) - 22
# # Transportation (Highways, Air transportation (airports), Parking facilities, Sea and inland port facilities) - 48/49

# # Transposes dataframe and rename columns
df = df.transpose()
df.columns = df.iloc[0]
df = df[1:]
df.reset_index(inplace=True)
df.rename(columns = {'Expenditure1':'State Expenditure', 
"index" : "State", "Utility expenditure" : "Utilities"}, inplace = True)

for col in [col for col in df.columns]:
    if col != "State":
        df[col] = df[col].str.replace(',','')
        df[col] = df[col].astype(int)

df["Health and Social Services"] = df[social].sum(axis=1)
df["Education Related"] = df[educ].sum(axis=1)
df["Public Administration"] = df[educ].sum(axis=1)
df["Transportation"] = df[transport].sum(axis=1)
df.drop(columns = social + educ + govt + transport, inplace=True)

required_col_names = [col for col in df.columns[1:]]

for col in [col for col in required_col_names]:
    if col == "State Expenditure":
        df["State Total as % of US Total"] = \
            df.apply(lambda x : (x[col] / int(df.loc[df["State"] ==
                "United States", col])) * 100, axis = 1)

    else:
        df[col + " (State as % of US)"] = \
            df.apply(lambda x : (x[col] / int(df.loc[df["State"] ==
                "United States", col])) * 100, axis = 1)

        df[col + " (% of Total Expenditure)"] = (df[col] /
        df["State Expenditure"] * 100)

df.set_index(["State"], inplace = True)

df

Description,State Expenditure,Utilities,Health and Social Services,Education Related,Public Administration,Transportation,State Total as % of US Total,Utilities (State as % of US),Utilities (% of Total Expenditure),Health and Social Services (State as % of US),Health and Social Services (% of Total Expenditure),Education Related (State as % of US),Education Related (% of Total Expenditure),Public Administration (State as % of US),Public Administration (% of Total Expenditure),Transportation (State as % of US),Transportation (% of Total Expenditure)
State,,,,,,,,,,,,,,,,,
Alaska,15076507,662585,3432117,3123400,3123400,1871269,0.356387,0.259708,4.394818,0.300677,22.764670,0.272825,20.717000,0.272825,20.717000,0.758438,12.411821
Alabama,52646924,3328580,16217017,15342284,15342284,3035451,1.244498,1.304678,6.322459,1.420720,30.803351,1.340129,29.141843,1.340129,29.141843,1.230288,5.765676
Arkansas,28980661,997150,9267271,8981653,8981653,1954279,0.685062,0.390845,3.440743,0.811875,31.977431,0.784536,30.991885,0.784536,30.991885,0.792082,6.743390
Arizona,71488388,6437486,19448164,19820840,19820840,3892365,1.689884,2.523252,9.004939,1.703790,27.204648,1.731326,27.725957,1.731326,27.725957,1.577601,5.444751
California,678462621,47236140,203249197,161448285,161448285,29738225,16.037890,18.514786,6.962232,17.805999,29.957317,14.102306,23.796195,14.102306,23.796195,12.053099,4.383178
Colorado,74023492,4609612,15038879,20290185,20290185,4807637,1.749810,1.806794,6.227229,1.317507,20.316360,1.772322,27.410467,1.772322,27.410467,1.948567,6.494745
Connecticut,45285851,1423391,6582163,14260766,14260766,2177743,1.070493,0.557916,3.143125,0.576642,14.534701,1.245660,31.490555,1.245660,31.490555,0.882654,4.808882
Delaware,13311734,611023,3443842,4494531,4494531,767628,0.314670,0.239498,4.590108,0.301704,25.870724,0.392592,33.763678,0.392592,33.763678,0.311125,5.766552
Florida,208234166,12233643,50051526,50274090,50274090,16427556,4.922359,4.795127,5.874945,4.384851,24.036174,4.391379,24.143055,4.391379,24.143055,6.658197,7.888982


In [4]:
# CLEANS CENSUS POPULATION

pop_df = pd.read_csv("us_census_population.csv")
pop_df = pop_df.iloc[:,0:2]
pop_df.columns = ["State", "2020 Census Population"]
pop_df['State'] = pop_df['State'].str.strip()
pop_df = pop_df[pop_df["State"].isin(STATE_NAMES_AND_UNITED_STATES)]

pop_df["2020 Census Population"] = pop_df["2020 Census Population"].str.replace(',','')
pop_df["2020 Census Population"] = pop_df["2020 Census Population"].astype(int)

pop_df.set_index("State", inplace=True)

pop_df


,2020 Census Population
State,
United States,331449520
Alabama,5024356
Alaska,733378
Arizona,7151507
Arkansas,3011555
California,39538245
Colorado,5773733
Connecticut,3605942
Delaware,989957


In [5]:
# CLEANS CENSUS POVERTY

poverty_df = pd.read_csv("us_poverty_by_state.csv")
poverty_df = poverty_df.iloc[:,0:2]
poverty_df.columns = ["State", "3-Year Average Poverty Rate (2018-2020)"]
poverty_df['State'] = poverty_df['State'].str.strip()
poverty_df = poverty_df[poverty_df["State"].isin(STATE_NAMES_AND_UNITED_STATES)]

poverty_df["3-Year Average Poverty Rate (2018-2020)"] = poverty_df["3-Year Average Poverty Rate (2018-2020)"].astype(float)

poverty_df.set_index("State", inplace=True)

poverty_df

,3-Year Average Poverty Rate (2018-2020)
State,
United States,11.2
Alabama,14.6
Alaska,12.2
Arizona,11.2
Arkansas,14.7
California,11.0
Colorado,9.3
Connecticut,9.9
Delaware,8.1


In [6]:
# COMBINES MULTIPLE DATAFRAMES INTO ONE

df_lst = [(poverty_df, []), (pop_df, [])]

combine_dataframes_by_state(df, df_lst)

,State Expenditure,Utilities,Health and Social Services,Education Related,Public Administration,Transportation,State Total as % of US Total,Utilities (State as % of US),Utilities (% of Total Expenditure),Health and Social Services (State as % of US),Health and Social Services (% of Total Expenditure),Education Related (State as % of US),Education Related (% of Total Expenditure),Public Administration (State as % of US),Public Administration (% of Total Expenditure),Transportation (State as % of US),Transportation (% of Total Expenditure),2020 Census Population,3-Year Average Poverty Rate (2018-2020)
State,,,,,,,,,,,,,,,,,,,
Alaska,15076507,662585,3432117,3123400,3123400,1871269,0.356387,0.259708,4.394818,0.300677,22.764670,0.272825,20.717000,0.272825,20.717000,0.758438,12.411821,733378,12.2
Alabama,52646924,3328580,16217017,15342284,15342284,3035451,1.244498,1.304678,6.322459,1.420720,30.803351,1.340129,29.141843,1.340129,29.141843,1.230288,5.765676,5024356,14.6
Arkansas,28980661,997150,9267271,8981653,8981653,1954279,0.685062,0.390845,3.440743,0.811875,31.977431,0.784536,30.991885,0.784536,30.991885,0.792082,6.743390,3011555,14.7
Arizona,71488388,6437486,19448164,19820840,19820840,3892365,1.689884,2.523252,9.004939,1.703790,27.204648,1.731326,27.725957,1.731326,27.725957,1.577601,5.444751,7151507,11.2
California,678462621,47236140,203249197,161448285,161448285,29738225,16.037890,18.514786,6.962232,17.805999,29.957317,14.102306,23.796195,14.102306,23.796195,12.053099,4.383178,39538245,11.0
Colorado,74023492,4609612,15038879,20290185,20290185,4807637,1.749810,1.806794,6.227229,1.317507,20.316360,1.772322,27.410467,1.772322,27.410467,1.948567,6.494745,5773733,9.3
Connecticut,45285851,1423391,6582163,14260766,14260766,2177743,1.070493,0.557916,3.143125,0.576642,14.534701,1.245660,31.490555,1.245660,31.490555,0.882654,4.808882,3605942,9.9
Delaware,13311734,611023,3443842,4494531,4494531,767628,0.314670,0.239498,4.590108,0.301704,25.870724,0.392592,33.763678,0.392592,33.763678,0.311125,5.766552,989957,8.1
Florida,208234166,12233643,50051526,50274090,50274090,16427556,4.922359,4.795127,5.874945,4.384851,24.036174,4.391379,24.143055,4.391379,24.143055,6.658197,7.888982,21538226,12.8


In [130]:
# CLEANS FUNDING DATA

raw_funding_df = pd.read_csv("2016_us_funding.csv")
raw_funding_df["code"] = raw_funding_df["code"].astype(str)

NAICS_SECTOR_CODES = {
    "Agriculture, Forestry, Fishing and Hunting" : ("11",),
    "Mining, Quarrying, and Oil and Gas Extraction" : ("21",),
    "Utilities" : ("22",),
    "Construction" : ("23",),
    "Manufacturing" : ("31", "32", "33",),
    "Wholesale Trade" : ("42",),
    "Retail Trade" : ("44", "45",),
    "Transportation and Warehousing" : ("48", "49",),
    "Information" : ("51",),
    "Finance and Insurance" : ("52",),
    "Real Estate and Rental and Leasing" : ("53",),
    "Professional, Scientific, and Technical Services" : ("54",),
    "Management of Companies and Enterprises" : ("55",),
    "Administrative and Support and Waste Management and Remediation Services" : ("56",),
    "Educational Services" : ("61",),
    "Health Care and Social Assistance" : ("62",),
    "Arts, Entertainment, and Recreation" : ("71",),
    "Accommodation and Food Services" : ("72",),
    "Other Services (except Public Administration)" : ("81",),
    "Public Administration (not covered in economic census)" : ("92",)
}

# Creates structure for funding dataframe
naics_sector_lst = [k for k in NAICS_SECTOR_CODES.keys()]
funding_df = pd.DataFrame(STATE_NAMES_AND_UNITED_STATES, columns=["State"])
funding_df = pd.concat([funding_df,pd.DataFrame(columns = naics_sector_lst)])

# Sums up funding for each category in each state and inputs values into funding_df
count = 0
for state_code, state in US_STATE_CODES.items():
    for sector, naics_code_tuple in NAICS_SECTOR_CODES.items():
        if len(naics_code_tuple) == 1:
            subset_df = raw_funding_df[(raw_funding_df["code"].apply(lambda x : x.startswith(naics_code_tuple[0]))) & (raw_funding_df["State"] == state_code)]
            sum_val = subset_df["amount"].sum()
        else:
            sum_val = 0
            for naics_code in naics_code_tuple:
                subset_df = raw_funding_df[(raw_funding_df["code"].apply(lambda x : x.startswith(naics_code))) & (raw_funding_df["State"] == state_code)]
                sum_val += subset_df["amount"].sum()
        
        funding_df.loc[funding_df["State"] == state, sector] = int(sum_val)
    count += 1
    if count == 2:
        break

funding_df

Alabama Public Administration (not covered in economic census) 1543306743.1100001
Alaska Public Administration (not covered in economic census) 1543306743.1100001
